In [3]:
import tensorflow as tf 
import warnings
warnings.filterwarnings('ignore')

In [4]:
from tensorflow.keras.models import load_model

In [6]:
model = load_model('./saved_model')

In [12]:
import os
from glob import glob
import matplotlib.pyplot as plt
import random
import cv2
import numpy as np
import matplotlib.gridspec as gridspec
import zlib
import itertools
import sklearn
from sklearn.mixture import GaussianMixture as GMM
import itertools
import scipy
import skimage
from skimage.transform import resize
import tensorflow as tf
from imutils.object_detection import non_max_suppression
from imutils import paths
import argparse
import imutils
%config Completer.use_jedi = False

In [11]:
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

def preprocess(img):
    ret_img=None
    candidates=[]
    x=np.zeros([350,500,3], np.uint8)
    kernal=np.ones((3,3),np.uint8)
    image = imutils.resize(img, width=min(400, img.shape[1]))
    x[50:(50+image.shape[0]),50:(50+image.shape[1])]=image
    image=x
    (rects, weights) = hog.detectMultiScale(image, winStride=(4,4),padding=(64,64), scale=1.05)
    rects = np.array([[x, y, x + w, y + h] for (x, y, w, h) in rects])
    pick = non_max_suppression(rects, probs=None, overlapThresh=0.95)
    for (xA, yA, xB, yB) in pick:


        #cv2.imwrite("output/image{}.png".format(couter),image[yA:yB,xA:xB])
        img=image[yA:yB,xA:xB]
        img=cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
        img=cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,57,20)
        #_,img=cv2.threshold(img,80,255,cv2.THRESH_BINARY)
        img=cv2.bitwise_not(img)
        img=cv2.dilate(img,kernal,iterations=1)
        candidates.append(img)
    biggest_size=0
    for img in candidates:
        size=img.shape[0]*img.shape[1]
        if size> biggest_size:
            ret_img=img
            biggest_size=size
    return ret_img
            

In [13]:
cap=cv2.VideoCapture("videos_to_test/test.mp4") # input the test video path here
while True:
    _,image=cap.read()

    person=preprocess(image)
    if  type (person)== type(None):
        continue
    cv2.imshow("person",person)
    image_to_pred = skimage.transform.resize(person, (1,75, 75, 3))
    img_arr = np.asarray(image_to_pred)
    y_pred = model.predict(img_arr)
    Y_pred_classes = np.argmax(y_pred,axis = 1)
    if Y_pred_classes[0] == 1:
        predection ="Abnormal"
    else:
        predection="Normal"
    cv2.putText(image,predection,(30,30),cv2.FONT_HERSHEY_COMPLEX,1,(255,0,0))
    cv2.imshow("predection",image)
    k=cv2.waitKey(10)
    if k == ord('q'):
        break
cv2.destroyAllWindows()

2021-07-12 10:37:12.437605: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-07-12 10:37:12.458855: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2194820000 Hz


In [18]:
def get_avg_preds(vid_path):
    cap = cv2.VideoCapture(vid_path) # input the test video path here
    total_preds = []
    
    while cap.isOpened():
        ret, image = cap.read()
        
        if not ret:
            print("Can't receive frame (stream end?). Exiting ...")
            break
            
        person = preprocess(image)
        
        if type(person) == type(None):
            continue
            
        image_to_pred = skimage.transform.resize(person, (1,75, 75, 3))
        img_arr = np.asarray(image_to_pred)
        
        y_pred = model.predict(img_arr)
        Y_pred_classes = np.argmax(y_pred,axis = 1)
        
        if Y_pred_classes[0] == 1:
            predection ="Abnormal"
            total_preds.append(1)
        else:
            predection="Normal"
            total_preds.append(0)

        k = cv2.waitKey(10)
        if k == ord('q'):
            break
            
    cv2.destroyAllWindows()
    
    count = 0
    for i in total_preds:
        if i == 1:
            count += 1
    if len(total_preds) == 0:
        print("can't detect persons in this video")
        break
    if count >= (len(total_preds) / 2):
        return 'Abnormal'
    else:
        return 'Noraml'

In [19]:
prediction = get_avg_preds("videos_to_test/test.mp4")

Can't receive frame (stream end?). Exiting ...


In [20]:
prediction

'Noraml'